In [13]:
from datasets import load_dataset
dataset = load_dataset("not-lain/wikipedia")

In [14]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 3000
    })
})

In [15]:
from sentence_transformers import SentenceTransformer
ST = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1")

In [16]:
def embed(batch):
    information=batch["text"]
    return {"embeddings":ST.encode(information)}
dataset=dataset.map(embed, batched=True, batch_size=16)

In [26]:
dataset.push_to_hub("Venomsnaker/wikipedia", private=True)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Venomsnaker/wikipedia/commit/5df37b2e5091cbde0e195f0e1b4d3b5716ea62dc', commit_message='Upload dataset', commit_description='', oid='5df37b2e5091cbde0e195f0e1b4d3b5716ea62dc', pr_url=None, pr_revision=None, pr_num=None)

In [28]:
from datasets import load_dataset
dataset = load_dataset("Venomsnaker/wikipedia")

ReadTimeout: (ReadTimeoutError("HTTPSConnectionPool(host='cdn-lfs-us-1.huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 9ee277c9-4121-4c7d-9aae-77ceffab22a0)')

In [ ]:
data = dataset["train"]
data = data.add_faiss_index("embeddings")

In [ ]:
def search(query: str, k: int = 3 ):
    """a function that embeds a new query and returns the most probable results"""
    embedded_query = ST.encode(query) # embed new query
    scores, retrieved_examples = data.get_nearest_examples( # retrieve results
        "embeddings", embedded_query, # compare our new embedded query with the dataset embeddings
        k=k # get only top k results
    )
    return scores, retrieved_examples

In [ ]:
scores , result = search("anarchy", 4 ) 
result['title']
['Anarchism', 'Anarcho-capitalism', 'Community', 'Capitalism']

In [ ]:
print(result["text"][0])